# Machine learning of m6A sams RNAs with Nanopore

In [ ]:
# packages
import mkl
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from hyperopt import hp, tpe, Trials, fmin, space_eval

In [ ]:
# Processers
mkl.set_num_threads(20)

In [ ]:
# load current data
with open('fast5_current_m6A_sams-345_100nt.pickle', 'rb') as f:
    current = pickle.load(f)
print(current.shape)

In [ ]:
# specify ranges
length = range(0,303)
df = current[((current['sams'] == 'sams-3_E2/E3L') | (current['sams'] == 'sams-5_E2L/E3L')) & ((current['RNA'] == 'unm') | (current['RNA'] == 'm6A'))]

# unm = 0, m6A = 1
Y = df['RNA']
Y = Y.str.replace('unm','0').str.replace('m6A','1').values
Y = np.array(list(map(int, Y)))

# current
X = df.iloc[:,length].values

# training data and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0, test_size=0.2)

# downsampling
X_train_m6A = X_train[np.where(Y_train == 1)]
X_train_unm = X_train[np.where(Y_train == 0)]
X_train_unm = X_train_unm[np.random.choice(len(X_train_unm), len(X_train_m6A), replace=False)]
X_train = np.concatenate([X_train_unm, X_train_m6A])
Y_train = np.concatenate([np.zeros(len(X_train_m6A), dtype=int), np.ones(len(X_train_m6A), dtype=int)])

In [ ]:
# data set for vivo
X_vivo3E2E3L = current[(current['sams'] == 'sams-3_E2/E3L') & (current['RNA'] == 'vivo')].iloc[:,length].values
X_vivo3retained = current[(current['sams'] == 'sams-3_retained') & (current['RNA'] == 'vivo')].iloc[:,length].values
X_vivo4E2E3L = current[(current['sams'] == 'sams-4_E2/E3L') & (current['RNA'] == 'vivo')].iloc[:,length].values
X_vivo4E2LE3L = current[(current['sams'] == 'sams-4_E2L/E3L') & (current['RNA'] == 'vivo')].iloc[:,length].values
X_vivo4retained = current[(current['sams'] == 'sams-4_retained') & (current['RNA'] == 'vivo')].iloc[:,length].values
X_vivo5E2LE3L = current[(current['sams'] == 'sams-5_E2L/E3L') & (current['RNA'] == 'vivo')].iloc[:,length].values

In [ ]:
# load models
# Decision tree
from sklearn.tree import DecisionTreeClassifier
# Random forest
from sklearn.ensemble import RandomForestClassifier
# Logistic regression
from sklearn.linear_model import LogisticRegression
# KNN
from sklearn.neighbors import KNeighborsClassifier
# SVC
from sklearn.svm import SVC
# AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
# GaussianNB
from sklearn.naive_bayes import GaussianNB
# LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# MLPClassifier
from sklearn.neural_network import MLPClassifier
# XGBoost
import xgboost as xgb
# LightGBM
import lightgbm as lgbm

# Compare classifiers by tuned parameters

In [ ]:
# scaling?
withScaling = {'SVM',
              'LogisticRegression',
              'KNeighbors',
              'MLP'
             }

# Classifiers
classifiers = {'GradientBoostingClassifier': GradientBoostingClassifier,
               'XGBoost': xgb.XGBClassifier,
               'LightGBM': lgbm.LGBMClassifier,
               'DecisionTree': DecisionTreeClassifier,
               'RandomForest': RandomForestClassifier,
               'SVM': SVC,
               'LogisticRegression': LogisticRegression,
               'KNeighbors': KNeighborsClassifier,
               'MLP': MLPClassifier
              }



    
# Parameters
params = {'GradientBoostingClassifier': {'learning_rate': hp.uniform('learning_rate', 0.01, 2),
                                         'max_depth': hp.choice('max_depth', range(1,40)),
                                         'min_samples_leaf': hp.choice('min_samples_leaf', range(1,20)),
                                         'max_features': hp.uniform('max_features', 0.01, 1)
                                         },
          
          'XGBoost': {'learning_rate': hp.uniform('learning_rate', 0.01, 2),
                      'max_depth': hp.choice('max_depth', np.arange(1, 40, 1, dtype=int)),
                      'min_child_weight': hp.choice('min_child_weight', np.arange(1, 10, 1, dtype=int)),
                      'colsample_bytree': hp.uniform('colsample_bytree', 0.2, 1),
                      'subsample': hp.uniform('subsample', 0.2, 1),
                      'n_estimators': 100
                     },
          
          'LightGBM': {'learning_rate': hp.uniform('learning_rate', 0.01, 2),
                       'max_depth': hp.choice('max_depth', np.arange(1, 40, 1, dtype=int)),
                       'min_child_weight': hp.choice('min_child_weight', np.arange(1, 10, 1, dtype=int)),
                       'colsample_bytree': hp.uniform('colsample_bytree', 0.2, 1),
                       'subsample': hp.uniform('subsample', 0.2, 1),
                       'n_estimators': 100
                      },
          
          'DecisionTree': {'max_depth': hp.choice('max_depth', np.arange(1, 40, 2, dtype=int)),
                           'max_features': hp.choice('max_features', np.arange(1, 20, 1, dtype=int)),
                           'min_samples_split': hp.choice('min_samples_split', np.arange(2, 20, 1, dtype=int)),
                           'min_samples_leaf': hp.choice('min_samples_leaf', np.arange(1, 20, 1, dtype=int))
                          },
          
          'RandomForest': {'max_depth': hp.choice('max_depth', np.arange(1, 40, 1, dtype=int)),
                           'max_features': hp.choice('max_features', np.arange(1, 20, 1, dtype=int)),
                           'min_samples_split': hp.choice('min_samples_split', np.arange(2, 20, 1, dtype=int)),
                           'min_samples_leaf': hp.choice('min_samples_leaf', np.arange(1, 20, 1, dtype=int))
                          },
          
          'SVM': {'C': hp.loguniform('C', -6, 2),
                  'gamma': hp.loguniform('gamma', -6, 2),
                  'kernel': hp.choice('kernel', ['linear', 'rbf', 'poly']),
                  'cache_size': 10000
                 },
          
          'LogisticRegression': {'C': hp.uniform('C', 0.00001, 1000),
                                 'random_state': hp.choice('random_state', np.arange(1, 100, 1, dtype=int))
                                },
          
          'KNeighbors': {'weights': hp.choice('weights', ['uniform','distance']),
                         'leaf_size': hp.choice('leaf_size', np.arange(5, 50, 5, dtype=int)),
                         'n_neighbors': hp.choice('n_neighbors', np.arange(1, 30, 1, dtype=int)),
                         'p': hp.choice('p', np.arange(1, 3, 1, dtype=int))
                        },
          
          'MLP': {'alpha': hp.loguniform('alpha', np.log(0.0001), np.log(0.9)),
                  'hidden_layer_sizes': hp.choice('hidden_layer_sizes', np.arange(100, 1000, 50, dtype=int)),
                  'learning_rate': hp.choice('learning_rate', ['constant','adaptive']),
                  'activation': 'relu',
                  'solver': 'adam'
                 }
         
         }


In [ ]:
# fitting parameters

# maximize accuracy score
def objective(args):
    
    clf = classifier(**args)
    clf.fit(X_train, Y_train)
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    scoreVal = cross_validate(clf, X_train, Y_train, cv=kf)
    return -scoreVal['test_score'].mean()



# tune classifier
def tuneClassifier(name, classifier, params):
    
    # scaling
    if name in withScaling:
        
        # scaling
        stdsc = StandardScaler()
        X_train = stdsc.fit_transform(X_train)
        X_test = stdsc.transform(X_test)
        X_vivo3E2E3L = stdsc.transform(X_vivo3E2E3L)
        X_vivo3retained = stdsc.transform(X_vivo3retained)
        X_vivo4E2E3L = stdsc.transform(X_vivo4E2E3L)
        X_vivo4E2LE3L = stdsc.transform(X_vivo4E2LE3L)
        X_vivo4retained = stdsc.transform(X_vivo4retained)
        X_vivo5E2LE3L = stdsc.transform(X_vivo5E2LE3L)
        
    
    # save steps
    trials = Trials()

    # tuning
    best = fmin(
        objective,
        params,
        algo=tpe.suggest,
        max_evals=100,
        trials=trials,
        verbose=1
    )

    # best params
    clf = classifier(**space_eval(params, best))
    clf.fit(X_train, Y_train)

    # accuracy
    accuracyTrain = clf.score(X_train, Y_train)
    accuracyTest = clf.score(X_test, Y_test)

    # vitro
    predictVitroUnm = clf.predict(X_test[np.where(Y_test == 0)])
    scoreVitroUnm = len(predictVitroUnm[predictVitroUnm == 1])/len(predictVitroUnm)
    predictVitrom6A = clf.predict(X_test[np.where(Y_test == 1)])
    scoreVitrom6A = len(predictVitrom6A[predictVitrom6A == 1])/len(predictVitrom6A)

    # vivo
    scoreVivo3E2E3L = np.count_nonzero(clf.predict(X_vivo3E2E3L))/len(X_vivo3E2E3L)
    scoreVivo3retained = np.count_nonzero(clf.predict(X_vivo3retained))/len(X_vivo3retained)
    scoreVivo4E2E3L = np.count_nonzero(clf.predict(X_vivo4E2E3L))/len(X_vivo4E2E3L)
    scoreVivo4E2LE3L = np.count_nonzero(clf.predict(X_vivo4E2LE3L))/len(X_vivo4E2LE3L)
    scoreVivo4retained = np.count_nonzero(clf.predict(X_vivo4retained))/len(X_vivo4retained)
    scoreVivo5E2LE3L = np.count_nonzero(clf.predict(X_vivo5E2LE3L))/len(X_vivo5E2LE3L)
    
    # save the model
    with open('Tuned_Model_' + name + '.pickle', 'wb') as f:
        pickle.dump(clf, f)
    
    # save feature importance 
    if hasattr(clf, 'feature_importances_'):
        np.savetxt('Importance_' + name + '.csv', clf.feature_importances_, delimiter=',')

    # save scores    
    f.write('%s,'\
            '%s, %s, %s, %s,'\
            '%s, %s, %s, %s, %s, %s'
            % (name,
               accuracyTrain, accuracyTest, scoreVitroUnm, scoreVitrom6A,
               scoreVivo3E2E3L, scoreVivo3retained, scoreVivo4E2E3L, scoreVivo4E2LE3L, scoreVivo4retained, scoreVivo5E2LE3L
            ))

In [ ]:
# output the accuracy and the fractions of modified bases
out_path = base_name + '.txt'
with open(out_path, mode='w') as f:


    # output results    
    f.write('Name, accuracyTrain, accuracyTest, scoreVitroUnm, scoreVitrom6A,'\
            'scoreVivo3E2E3L, scoreVivo3retained, scoreVivo4E2E3L, scoreVivo4E2LE3L, scoreVivo4retained, scoreVivo5E2LE3L\n'\
            
            'Size, %s, %s, %s, %s,'\
            '%s, %s, %s, %s, %s, %s\n'\
            % (len(X_train), len(X_test), len(predictVitroUnm), len(predictVitrom6A),
               len(X_vivo3E2E3L), len(X_vivo3retained), len(X_vivo4E2E3L), len(X_vivo4E2LE3L), len(X_vivo4retained), len(X_vivo5E2LE3L)
            ))



In [ ]:
with open('test.txt', mode='w') as f:

    
    # output results    
    f.write('Name, accuracyTrain, accuracyTest, scoreVitroUnm, scoreVitrom6A\n'\
            'Size, 1, 2, 3, 4\n'
            )
    
    
    for i in range(10):
        f.write('%s, %s, %s, %s, %s\n' % (i, i+1, i+2, i+3, i+4))
